In [1]:
from google.colab import drive
drive.mount('/content/drive')

from tqdm import tqdm
import pandas as pd
import numpy as np
import re

Mounted at /content/drive


In [3]:
data = pd.read_csv('/content/drive/MyDrive/RMTC/Август/1-7 августа/product_random_description.csv', sep=';')
data.head()

,id,brand_name,article,product_name,product_type,product_category,photo,famaga_photo,description_ru,description_de,description_und
0,118048,Bernstein,6018200002,SLC-M-024-11/11-R4,Защитный выключатель блокировки,Переключатели,https://www.bernstein.eu/fileadmin/img/produkt...,NaN,NaN,NaN,"SLC-M-024-11/11-R4, Safety switch: blocking; S..."
1,118053,Bernstein,6018200007,SLC-F-024-20/20-R4,Защитный выключатель блокировки,Переключатели,https://www.bernstein.eu/fileadmin/img/produkt...,NaN,NaN,NaN,model SLC-F-024-20/20-R4
2,118054,Bernstein,6018200008,SLC-F-024-11/20-R4,Защитный выключатель блокировки,Переключатели,https://www.bernstein.eu/fileadmin/img/produkt...,NaN,NaN,NaN,1. Safety interlock switch SLC-F-024-11/20-R4 ...
3,118074,Bernstein,6018119045,SLK-F-UC-55-R1-A0-L0-0,NaN,Переключатели,https://www.bernstein.eu/fileadmin/_migrated/p...,NaN,переключатель \nпоследующая модель для 6018119...,"SLK-F-UC-55-R1-A0-L0-0 , Sicherheitschalter [ ...",BERNSTEIN GERMANY limit sw slk-FVTU24UC55ARX H...
4,118076,Bernstein,6018119047,SLK-M-UC-55-R0-A0-L0-0,NaN,Переключатели,https://www.bernstein.eu/fileadmin/_migrated/p...,NaN,выключатель SLK-M-UC-55-R0-A0-L0-0 A,NaN,SLK-M-UC-55-R0-A0-L0-0 A


In [4]:
data['description_und'] = data['description_und'].fillna('KI')

In [5]:
def only_words(x):
  result = re.findall(r'\b[a-zA-Z]+\b(?!-)[ ,.;:]', x)
  return ' '.join(result)

data['description_und'] = data['description_und'].apply(only_words)
data['description_und'] = data['description_und'].replace('', 'KI')
data['brand_name'] = data['brand_name'].apply(lambda x: x.lower())
data['description_und'] = data['description_und'].apply(lambda x: x.lower())
data['description_und'] = data.apply(lambda row: row['description_und'].replace(row['brand_name'], ''), axis=1)
data['description_und'].head()

,description_und
0,"safety switch: blocking; slc; contacts: no, s..."
1,model
2,safety interlock switch pc.
3,"germany limit sw ho, sicherung, l sicher..."
4,ki


In [15]:
description = data['description_und'].to_list()
for i in range(len(description)):
  description[i] = description[i].replace('\t', ' ')
  description[i] = description[i].replace('  ', ' ')
  description[i] = description[i].replace('   ', ' ')
  description[i] = description[i].lstrip().rstrip()

In [13]:
set_description = set()
all_text = list()

for y in tqdm(description):
    text = [x.lstrip().rstrip() for x in re.split(": |; |, |\. |:|;|,|\.", y) if len(x.lstrip().rstrip()) > 1] # получаем разбивку описаний
    all_text.extend(text)
    set_description.update(set(text))


100%|██████████| 356960/356960 [00:01<00:00, 265859.18it/s]


In [18]:
description_count = dict()
for key in tqdm(set_description):
  description_count[key] = all_text.count(key)

100%|██████████| 320524/320524 [1:59:02<00:00, 44.88it/s]


In [19]:
count = pd.DataFrame(list(description_count.items()), columns=['Description', 'Value'])

In [23]:
count.sort_values(by='Value', ascending=False)

,Description,Value
303741,ki,98208
58045,nr,4673
207814,ursprungsland,3685
173811,mm,2549
161788,type,2293
...,...,...
118459,sicherheitsventil magnetventil sicherheitsvent...,1
118462,und sensor isemax,1
118464,highly resistant,1
118466,filter element separator brenngas verdichtertyp,1


In [24]:
count.sort_values(by='Value', ascending=False).to_excel('Частотность ключевых слов_очищенная версия.xlsx', index=False)

In [ ]:
print(set(data_with_photolinks['brand_name'].to_list()))

Отдельно указать совпадениям по типу товара

* Совпадение с процентом
* Если в названии есть характеристика, то это точно не тип товара
* Является ли типом товаров
* Вообще не является типом товара - на подумать


2. Поиск соответствия ключевого слова по названию фото товара (как я предлогал вытаскивать из фото название и тип)